# Bloodhound Playbook - Top Tens
-----------------------------------------


## Import Libraries

In [ ]:
from py2neo import Graph

## Initialize BloodHound neo4j Database Connection

In [ ]:
g = Graph("bolt://127.0.0.1:7687", auth=("neo4j", "neo4j"))
g

## Retrieve Top Ten Users With Most Local Admin Rights

Now we’ll construct the Cypher query that tells us the name of the ten users with most local admin rights

In [ ]:
local_rights_df = g.run("""
MATCH (n:User),(m:Computer), (n)-[r:AdminTo]->(m) WHERE NOT n.name STARTS WITH 'ANONYMOUS LOGON' AND NOT n.name='' 
WITH n, count(r) as rel_count order by rel_count desc MATCH p=(m)<-[r:AdminTo]-(n) 
RETURN n.name, count(r)
""").to_data_frame()

In [ ]:
local_rights_df[:10]

## Retrieve Top Ten Users With Most Admin Rights

In [ ]:
most_rights_df = g.run("""
MATCH (U:User)-[r:MemberOf|:AdminTo*1..]->(C:Computer)
WITH U.name as n, COUNT(DISTINCT(C)) as c 
RETURN n,c
ORDER BY c DESC
""").to_data_frame()

In [ ]:
most_rights_df[45:55]

## Retrieve Top Ten Computers With Most Admins

In [ ]:
comp_admin_df = g.run("""
MATCH (n:User),(m:Computer), (n)-[r:AdminTo]->(m) WHERE NOT n.name STARTS WITH 'ANONYMOUS LOGON' AND NOT n.name='' 
WITH m, count(r) as rel_count order by rel_count desc LIMIT 10 MATCH p=(m)<-[r:AdminTo]-(n) 
RETURN DISTINCT m.name, count(r)
""").to_data_frame()

In [ ]:
comp_admin_df

## Retrieve Top Ten Users With Most Sessions

In [ ]:
user_sessions_df = g.run("""
MATCH (n:User),(m:Computer), (n)<-[r:HasSession]-(m) WHERE NOT n.name STARTS WITH 'ANONYMOUS LOGON' AND NOT n.name='' 
WITH m, count(r) as rel_count order by rel_count desc MATCH p=(m)-[r:HasSession]->(n) 
RETURN n.name,count(r) AS name_count
ORDER BY name_count DESC
""").to_data_frame()

In [ ]:
user_sessions_df[:10]